In [1]:
import requests
import transformers
from transformers import AutoTokenizer
import re
import string
import pandas as pd
# import nltk
import torch

from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama
from crewai import Agent , Task ,Crew , Process

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/llama3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [217]:
model = Ollama(model= "llama3.1:latest")

In [218]:
model

Ollama(model='llama3.1:latest')

In [9]:
!pip freeze

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
alembic==1.13.2
annotated-types==0.7.0
anyio==4.4.0
appdirs==1.4.4
appnope==0.1.4
asgiref==3.8.1
asttokens==2.4.1
async-timeout==4.0.3
attrs==24.2.0
auth0-python==4.7.2
backoff==2.2.1
bcrypt==4.2.0
beautifulsoup4==4.12.3
boto3==1.35.19
botocore==1.35.19
build==1.2.2
cachetools==5.5.0
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.3.2
chroma-hnswlib==0.7.3
chromadb==0.4.24
click==8.1.7
cohere==5.9.2
coloredlogs==15.0.1
comm==0.2.2
crewai==0.55.2
cryptography==43.0.1
dataclasses-json==0.6.7
debugpy==1.8.5
decorator==5.1.1
Deprecated==1.2.14
distro==1.9.0
docstring_parser==0.16
embedchain==0.1.121
exceptiongroup==1.2.2
executing==2.1.0
fastapi==0.114.2
fastavro==1.9.7
filelock==3.16.0
flatbuffers==24.3.25
frozenlist==1.4.1
fsspec==2024.9.0
google-api-core==2.19.2
google-auth==2.34.0
google-cloud-aiplatform==1.66.0
google-cloud-bigquery==3.25.0
google-cloud-core==2.4.1
google-cloud-resource-manager==1.12.5
google-cloud-storage

In [219]:
import json

js = json.load(open("json_data/transcripts_2024.json"))

In [220]:
df = pd.DataFrame()

for i in js:
    df_sub = pd.DataFrame(i)
    df = pd.concat([df,df_sub])

In [221]:
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/llama3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [222]:
def clean_text(text):
  text = re.sub(r'[^A-Za-z0-9\s.\(\)[\]{\}]+', '' , text)
  text = text.lower()
  text = " ".join(text.split())
  return text


def count_tokens(text):
  tokens = tokenizer.encode(text , add_special_tokens = True)
  return(len(tokens))

In [196]:
df['clean_content'] = df['content'].apply(clean_text)
df['count_tokens'] = df['clean_content'].apply(count_tokens)

In [204]:
df.reset_index(drop = True , inplace = True)

In [205]:
df[df['count_tokens'] < 7000].head(40)

,symbol,quarter,year,date,content,clean_content,count_tokens
4368,BORR,2,2024,2024-08-15 18:47:08,Operator: Good day and thank you for standing ...,operator good day and thank you for standing b...,6997
4369,MGNI,2,2024,2024-08-07 20:20:24,"Operator: Good day, and welcome to the Magnite...",operator good day and welcome to the magnite s...,6995
4370,SHOO,2,2024,2024-07-31 15:09:12,Operator: Good day and thank you for standing ...,operator good day and thank you for standing b...,6990
4371,JACK,1,2024,2024-02-21 22:12:02,"Operator: Thank you for standing by, and welco...",operator thank you for standing by and welcome...,6985
4372,CW,1,2024,2024-05-02 00:00:00,"Operator: Welcome, everyone, to the Curtiss-Wr...",operator welcome everyone to the curtisswright...,6979
4373,ROLL,1,2024,2023-08-04 18:03:07,"Operator: Greetings, and welcome to RBC Bearin...",operator greetings and welcome to rbc bearings...,6978
4374,NATL,2,2024,2024-08-14 13:17:07,"Operator: Good day, and welcome to the NCR Atl...",operator good day and welcome to the ncr atleo...,6978
4375,U,1,2024,2024-05-09 20:20:24,Daniel Amir: Welcome to Unity's First Quarter ...,daniel amir welcome to unitys first quarter 20...,6976
4376,SIG,3,2024,2023-12-05 11:12:02,"Operator: Good morning, and welcome to the Sig...",operator good morning and welcome to the signe...,6976
4377,UNF,1,2024,2024-01-03 10:55:03,"Operator: Greetings, and welcome to the UniFir...",operator greetings and welcome to the unifirst...,6975


In [177]:
from crewai import Agent
import os
import json

In [178]:
os.environ["OPENAI_API_BASE"] = 'https://api.groq.com/openai/v1'
os.environ["OPENAI_MODEL_NAME"] = 'llama3-8b-8192'  # Adjust based on available model
os.environ["OPENAI_API_KEY"] = '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'

In [210]:
# Choosing physicist bio from dataframe to acheive summary
clean_transcripts = pd.DataFrame(df.iloc[4375]).T
clean_transcripts

,symbol,quarter,year,date,content,clean_content,count_tokens
4375,U,1,2024,2024-05-09 20:20:24,Daniel Amir: Welcome to Unity's First Quarter ...,daniel amir welcome to unitys first quarter 20...,6976


In [229]:
# Create summarizer agent for bio
summarizer_bio = Agent(
    role='Summarizer',
    goal="Summarize the full text in no more than 2000 tokens. Include main challenges and successes in bullet points with metrics",
    verbose=True,
    memory=True,
    backstory=(
        "Equipped with advanced summarization techniques, "
        "the goal is to distill complex information into concise summaries."
    ),
    llm=model,
    allow_delegation=False  # No need for search_tool if performing local summarization
)

# Create summarizer agent for research
summarizer_research = Agent(
    role='Summarizer',
    goal='Accurately summarize the stock transcript with bullet points',
    verbose=True,
    memory=True,
    backstory=(
        "With a focus on distilling complex information into concise summaries, "
        "the goal is to provide clear and insightful summaries on various topics."
    ),
    allow_delegation=False,
)

In [230]:
# Define a function to create summary tasks
def create_summary_task(description_template, expected_output_template, stock_transcript, agent):
    description = description_template.format(stock_transcript=stock_transcript)
    expected_output = expected_output_template.format(stock_transcript=stock_transcript)
    return Task(
        description=description,
        expected_output=expected_output,
        agent=agent
    )

# Summary task for summarizing physicist bios
summary_task_summarizer_bio = create_summary_task(
    description_template="Summarize the transcript of {stock_transcript}.",
    expected_output_template="A concise summary of the transcript of {stock_transcript}.",
    stock_transcript=clean_transcripts,
    agent=summarizer_bio
)

# Summary task for summarizing physicist researches
summary_task_summarizer_research = create_summary_task(
    description_template="Summarize the transcript of {stock_transcript}.",
    expected_output_template="A concise summary of the transcript of {stock_transcript}.",
    stock_transcript=clean_transcripts,
    agent=summarizer_research
)

# Add the summary tasks to the crew
crew = Crew(
    agents=[summarizer_bio],
    tasks=[summary_task_summarizer_bio],
    process=Process.sequential
)

# Kick off the crew
crew_result = crew.kickoff()

# Initialize lists to store summary outputs
summary_bio_list = [''] * len(df)
#summary_research_list = [''] * len(df)

2024-09-14 16:14:43,254 - 8095521856 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer:

Daniel Amir: Welcome to Unity's First Quarter 2024 Earnings Call. Today is May 9th, 2024. I'm Daniel Amir, Unity's Chief Financial Officer. 

First quarter revenue was $434 million, up 25% year-over-year. Subscription and recurring revenues grew 32%, driven by increasing adoption of our platform across various industries.

Our platform gross margin expanded to 88% due to improved operational efficiency and better pricing strategies. Adjusted EBITDA was $123 million, representing a 28% increase from the same period last year.

The company's progress in expanding its platform capabilities, combined with an increased focus on customer success, contributed to these strong financial results.

Key takeaways:

* Revenue growth: 25% YoY
* Subscription and recurring revenues growth: 32%
* Platform gross margin expansion: 88%
* Adjusted EBITDA growth: 28%

Successes:

* Strong revenue growth
* Increasing a

In [231]:
crew_result.__dict__

{'raw': "Daniel Amir: Welcome to Unity's First Quarter 2024 Earnings Call. Today is May 9th, 2024. I'm Daniel Amir, Unity's Chief Financial Officer. \n\nFirst quarter revenue was $434 million, up 25% year-over-year. Subscription and recurring revenues grew 32%, driven by increasing adoption of our platform across various industries.\n\nOur platform gross margin expanded to 88% due to improved operational efficiency and better pricing strategies. Adjusted EBITDA was $123 million, representing a 28% increase from the same period last year.\n\nThe company's progress in expanding its platform capabilities, combined with an increased focus on customer success, contributed to these strong financial results.\n\nKey takeaways:\n\n* Revenue growth: 25% YoY\n* Subscription and recurring revenues growth: 32%\n* Platform gross margin expansion: 88%\n* Adjusted EBITDA growth: 28%\n\nSuccesses:\n\n* Strong revenue growth\n* Increasing adoption of the platform across various industries\n* Expanded pl

In [238]:
crew_result.raw.split('\n')

["Daniel Amir: Welcome to Unity's First Quarter 2024 Earnings Call. Today is May 9th, 2024. I'm Daniel Amir, Unity's Chief Financial Officer. ",
 '',
 'First quarter revenue was $434 million, up 25% year-over-year. Subscription and recurring revenues grew 32%, driven by increasing adoption of our platform across various industries.',
 '',
 'Our platform gross margin expanded to 88% due to improved operational efficiency and better pricing strategies. Adjusted EBITDA was $123 million, representing a 28% increase from the same period last year.',
 '',
 "The company's progress in expanding its platform capabilities, combined with an increased focus on customer success, contributed to these strong financial results.",
 '',
 'Key takeaways:',
 '',
 '* Revenue growth: 25% YoY',
 '* Subscription and recurring revenues growth: 32%',
 '* Platform gross margin expansion: 88%',
 '* Adjusted EBITDA growth: 28%',
 '',
 'Successes:',
 '',
 '* Strong revenue growth',
 '* Increasing adoption of the pl

In [163]:
crew_result.tasks_output[0].description.split('\n')

['Summarize the transcript of operator good morning and welcome to dun bradstreets financial results for the second quarter of 2024. [operator instructions] please note this event is being recorded. i would now like to turn the conference over to sean anthony vice president of fpa and investor relations. please go ahead. sean anthony thank you. good morning everyone and thank you for joining us for dun bradstreets financial results conference call for the second quarter of 2024. on the call today we have dun bradstreets ceo anthony jabbour and cfo bryan hipsher. anthony will begin with an overview of our second quarter results and then pass it to bryan for an indepth financial review. we will then finish up with qa and a few closing remarks. before we begin allow me to provide a disclaimer regarding forwardlooking statements. this call including the qa portion of the call may include forwardlooking statements related to the expected future results for our company and are therefore forw

In [68]:
clean_transcripts['clean_content'][0]

'suhasini chandramouli good afternoon and welcome to the apple q3 fiscal year 2024 earnings conference call. my name is suhasini chandramouli director of investor relations. todays call is being recorded. speaking first today is apples ceo tim cook and hell be followed by cfo luca maestri. after that well open the call to questions from analysts. please note that some of the information youll hear during our discussion today will consist of forwardlooking statements including without limitation those regarding revenue gross margin operating expenses other income and expense taxes capital allocation and future business outlook including the potential impact of macroeconomic conditions on the companys business and results of operations. these statements involve risks and uncertainties that may cause actual results or trends to differ materially from our forecast. for more information please refer to the risk factors discussed in apples most recently filed annual report on form 10k and th

In [92]:
lines = crew_result.tasks_output[0].description.split('\n')
lines

['Summarize the transcript of   symbol  quarter  year                 date  \\',
 '0   AAPL        3  2024  2024-08-01 19:36:04   ',
 '1   AAPL        2  2024  2024-05-02 20:30:10   ',
 '2   AAPL        1  2024  2024-02-01 20:17:02   ',
 '',
 '                                             content  \\',
 '0  Suhasini Chandramouli: Good afternoon and welc...   ',
 '1  Suhasini Chandramouli: Good Afternoon, and wel...   ',
 '2  Operator: Good day, and welcome to the Apple Q...   ',
 '',
 '                                       clean_content  count_tokens  ',
 '0  suhasini chandramouli good afternoon and welco...          9231  ',
 '1  suhasini chandramouli good afternoon and welco...          9470  ',
 '2  operator good day and welcome to the apple q1 ...          9029  .']

In [100]:
len(lines)

14

In [56]:
# Extract summary outputs from the crew result
if not isinstance(crew_result, str):
    # Split the crew result into lines
    # import ipdb;ipdb.set_trace()
    lines = crew_result.split('\n')

    # Find the indices where the outputs start and end for summary_bio and summary_research
    bio_start_index = lines.index("Final Answer for summary_transcript:")
    research_start_index = lines.index("Final Answer for summary_transcript:")

    # Extract the outputs and concatenate them properly
    summary_bio_list[0] = " ".join(lines[bio_start_index + 1:research_start_index])
    summary_research_list[0] = " ".join(lines[research_start_index + 1:])

# Append the summary results to the DataFrame
df['summary_bio'] = summary_bio_list
df['summary_research'] = summary_research_list

ValueError: 'Final Answer for summary_transcript:' is not in list